In [2]:
# 04_gold_anomalies.ipynb - VERSION CORRIGÉE
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("Gold_Anomalies").getOrCreate()

# CORRECTION: Utiliser les chemins locaux
BASE_PATH = "file:///tmp"
SILVER_NOAA = f"{BASE_PATH}/silver/noaa/"
SILVER_USGS = f"{BASE_PATH}/silver/usgs/"
GOLD_ANOM = f"{BASE_PATH}/gold/anomalies/"

# Créer le répertoire de sortie
import os
os.makedirs("/tmp/gold/anomalies", exist_ok=True)

# Charger les données Silver NOAA
print("📁 Chargement des données NOAA Silver...")
df_temp = spark.read.parquet(SILVER_NOAA)

# Ajouter mois et année
df_temp = df_temp.withColumn("month", F.month("date")).withColumn("year", F.year("date"))

print(f"📊 Données température chargées: {df_temp.count()} enregistrements")
df_temp.show(5)

# Calcul des anomalies de température par station
print("🔍 Calcul des anomalies de température...")

# Définir la fenêtre par station et mois
window_spec = Window.partitionBy("station_id", "month")

# Calculer la moyenne historique et écart-type par station et mois
df_anomalies = df_temp.withColumn("avg_temp_historical", F.avg("temp_c").over(window_spec)) \
                      .withColumn("std_temp_historical", F.stddev("temp_c").over(window_spec))

# Calculer l'anomalie (différence par rapport à la moyenne historique)
df_anomalies = df_anomalies.withColumn("temp_anomaly", 
                                       F.col("temp_c") - F.col("avg_temp_historical"))

# Identifier les anomalies significatives (> 2 écarts-types)
df_anomalies = df_anomalies.withColumn(
    "is_significant_anomaly",
    F.abs(F.col("temp_anomaly")) > (2 * F.col("std_temp_historical"))
)

# Ajouter un score d'anomalie normalisé
df_anomalies = df_anomalies.withColumn(
    "anomaly_score",
    F.when(F.col("std_temp_historical") > 0, 
           F.abs(F.col("temp_anomaly")) / F.col("std_temp_historical"))
     .otherwise(0)
)

print("📈 Statistiques des anomalies calculées:")
df_anomalies.select("temp_c", "avg_temp_historical", "temp_anomaly", "anomaly_score", "is_significant_anomaly").show(10)

# Charger les données USGS pour corrélation
print("\n📁 Chargement des données USGS Silver...")
try:
    df_seismic = spark.read.parquet(SILVER_USGS)
    print(f"📊 Données sismiques chargées: {df_seismic.count()} séismes")
    
    # Préparer les données sismiques pour jointure
    df_seismic_daily = df_seismic.groupBy("date").agg(
        F.count("*").alias("earthquake_count"),
        F.avg("magnitude").alias("avg_magnitude"),
        F.max("magnitude").alias("max_magnitude")
    )
    
    # Jointure avec les anomalies de température
    df_correlation = df_anomalies.join(
        df_seismic_daily, 
        on="date", 
        how="left"
    ).fillna(0, subset=["earthquake_count", "avg_magnitude", "max_magnitude"])
    
    print("🔗 Données jointes température-séismes")
    
except Exception as e:
    print(f"❌ Erreur chargement USGS: {e}")
    print("ℹ️  Utilisation des données température uniquement")
    df_correlation = df_anomalies

# Calcul des corrélations
print("\n📊 Analyse des corrélations...")

# Corrélation entre anomalies de température et activité sismique
if "earthquake_count" in df_correlation.columns:
    correlation_stats = df_correlation.filter(F.col("earthquake_count") > 0).agg(
        F.corr("temp_anomaly", "earthquake_count").alias("corr_temp_earthquake_count"),
        F.corr("anomaly_score", "earthquake_count").alias("corr_score_earthquake_count"),
        F.corr("temp_anomaly", "avg_magnitude").alias("corr_temp_avg_magnitude")
    ).collect()[0]
    
    print("📈 Coefficients de corrélation:")
    print(f"  • Anomalie température vs Nombre séismes: {correlation_stats['corr_temp_earthquake_count']:.3f}")
    print(f"  • Score anomalie vs Nombre séismes: {correlation_stats['corr_score_earthquake_count']:.3f}")
    print(f"  • Anomalie température vs Magnitude moyenne: {correlation_stats['corr_temp_avg_magnitude']:.3f}")

# Identifier les jours à haut risque
print("\n🚨 Identification des jours à haut risque...")

df_high_risk = df_correlation.filter(
    (F.col("is_significant_anomaly") == True) & 
    (F.col("earthquake_count") > 0)
).withColumn("risk_score", 
             F.col("anomaly_score") * F.col("earthquake_count") * F.col("avg_magnitude"))

# Classer par score de risque
df_high_risk_ranked = df_high_risk.orderBy(F.desc("risk_score"))

print("📋 Top 10 des jours à haut risque:")
df_high_risk_ranked.select(
    "date", "station_id", "temp_anomaly", "anomaly_score", 
    "earthquake_count", "avg_magnitude", "risk_score"
).show(10)

# Sauvegarder les résultats Gold
print("\n💾 Sauvegarde des données Gold...")

# Données d'anomalies détaillées
df_anomalies.write.mode("overwrite").partitionBy("year").parquet(f"{GOLD_ANOM}/temperature_anomalies")

# Données de corrélation et risque
df_correlation.write.mode("overwrite").partitionBy("year").parquet(f"{GOLD_ANOM}/correlation_analysis")

# Top des risques
df_high_risk_ranked.write.mode("overwrite").parquet(f"{GOLD_ANOM}/high_risk_days")

print(f"✅ Données Gold sauvegardées: {GOLD_ANOM}")

# Résumé final
print("\n🎉 === ANALYSE DES ANOMALIES TERMINÉE ===")
print(f"📈 Anomalies température: {df_anomalies.count()} enregistrements")
print(f"🚨 Jours à haut risque identifiés: {df_high_risk_ranked.count()}")

if df_high_risk_ranked.count() > 0:
    top_risk = df_high_risk_ranked.select("date", "risk_score").first()
    print(f"📅 Jour le plus risqué: {top_risk['date']} (score: {top_risk['risk_score']:.2f})")

# Visualisation des résultats
print("\n📊 RÉSUMÉ DES RÉSULTATS:")
print("Anomalies de température:")
df_anomalies.agg(
    F.count("*").alias("total_records"),
    F.avg("temp_anomaly").alias("avg_anomaly"),
    F.stddev("temp_anomaly").alias("std_anomaly"),
    F.sum(F.col("is_significant_anomaly").cast("int")).alias("significant_anomalies")
).show()

if "earthquake_count" in df_correlation.columns:
    print("Activité sismique corrélée:")
    df_correlation.filter(F.col("earthquake_count") > 0).agg(
        F.avg("earthquake_count").alias("avg_earthquakes_per_day"),
        F.max("earthquake_count").alias("max_earthquakes_per_day"),
        F.avg("avg_magnitude").alias("avg_magnitude")
    ).show()

📁 Chargement des données NOAA Silver...
📊 Données température chargées: 100 enregistrements
+-------------------+----------+-----------------+------------------+------------------+----+-----+
|               date|station_id|         latitude|         longitude|            temp_c|year|month|
+-------------------+----------+-----------------+------------------+------------------+----+-----+
|2024-03-16 00:00:00| STATION_0|40.48186609572492|-74.86755597998558| 11.23978774675729|2024|    3|
|2024-03-17 00:00:00| STATION_1|41.71198554111676|-74.15738771337645|16.633948794403523|2024|    3|
|2024-03-18 00:00:00| STATION_2|41.29907130307006|-74.99322556549149| 18.97464367553694|2024|    3|
|2024-03-19 00:00:00| STATION_3|40.97787938028068|-74.75964299884642|17.287862514585633|2024|    3|
|2024-03-20 00:00:00| STATION_4|39.91442034663697|-74.24808040111922| 12.29102717093713|2024|    3|
+-------------------+----------+-----------------+------------------+------------------+----+-----+
only sho